# Homework Assignment 5: Model Evaluation
As in the previous assignments, in this homework assignment you will continue your exploration of the [SWAN-SF Dataset](https://doi.org/10.7910/DVN/EBCFKM), described in the paper found [here](https://doi.org/10.1038/s41597-020-0548-x).


This assignment will utilize a copy of the extracted feature dataset we have been working with. The dataset has been processed by performing outlier clipping, z-score and range scaling, and forward feature selection to select 20 features. We are now going to utilize more than one partition worth of data, so for the z-score and range scaling, the mean, standard deviation, minimum, and maximum were calculated using data from both partitions so that a global scaling can be performed on each partition. 

---

## Step 1: Downloading the Data

This assignment will continue to only use [Partition 1](https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/EBCFKM/BMXYCB) and will add the use of [Partition 2](https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/EBCFKM/TCRPUD) as a testing set. 

---

For this assignment, cleaning, transforming, and normalization of the data has been completed using both partitions to find the various minimum, maximum, standard deviation, and mean values needed to perform these operations. Recall from lecture that we should not perform these operations on each partition individually, but as a whole as there may(will) be different values for these in different partitions. 

For example, if we perform simple range scaling on each partition individually and we see a range of 0 to 100 in one partition and 0 to 10 in another. After individual scaling the values with 100 in the first would be mapped to 1 just like the values that had 10 in the second. This can cause serious performance problems in your model, so I have made sure that the normalization was treated properly for you. 

Below you will find the full partitions and `toy` sampled data from each partition, where only 20 samples from each of our 5 classes have been included in the data.  

#### Full
- [Full Normalized Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/normalized_partition1ExtractedFeatures.csv)
- [Full Normalized Partition 2 feature dataset](http://dmlab.cs.gsu.edu/solar/data/normalized_partition2ExtractedFeatures.csv)

#### Toy
- [Toy Normalized Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/toy_normalized_partition1ExtractedFeatures.csv)
- [Toy Normalized Partition 2 feature dataset](http://dmlab.cs.gsu.edu/solar/data/toy_normalized_partition2ExtractedFeatures.csv)

Now that you have the two files, you should load each into a Pandas DataFrame using the [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) method. 

---

### Evaluation Metric

For each of the models we evaluate in this assignmnet, you will calculate the True Skill Statistic score using the test data from Partition 2 to determine which model performs the best for classifying the positive flaring class.

    True skill statistic (TSS) = TPR + TNR - 1 = TPR - (1-TNR) = TPR - FPR

Where:

    True positive rate (TPR) = TP/(TP+FN) Also known as recall or sensitivity
    True negative rate (TNR) = TN/(TN+FP) Also known as specificity or selectivity
    False positive rate (FPR) = FP/(FP+TN) = (1-TNR) Also known as fall-out or false alarm ratio


**Recall**

    True positive (TP)
    True negative (TN)
    False positive (FP)
    False negative (FN)
    
See [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix) for more information.

Below is a function implemented to provide your score for each model.

In [130]:
import os
import itertools
import pandas as pd
from pandas import DataFrame 
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import chi2

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split


In [131]:
def calc_tss(y_true=None, y_predict=None):
    """
    Calculates the true skill score for binary classification based on the output of the confusion
    table function
    """
    scores = confusion_matrix(y_true, y_predict).ravel()
    TN, FP, FN, TP = scores
    print('TN={0}\tFP={1}\tFN={2}\tTP={3}'.format(TN, FP, FN, TP))
    tp_rate = TP / float(TP + FN) if TP > 0 else 0  
    fp_rate = FP / float(FP + TN) if FP > 0 else 0
    
    return tp_rate - fp_rate

As in the previous assignment, we will be utilizing a binary classification of our 5 class dataset. So, below is the helper function to change our class labels from the 5 class target feature to the binary target feature. The function is implemented to take a dataframe (e.g. our `abt`) and prepares it for a binary classification by merging the `X`- and `M`-class samples into one group, and the rest (`NF`, `B`, and `C`) into another group, labeled with `1`s and `0`s, respectively.

In [132]:
def dichotomize_X_y(data: pd.DataFrame):
    """
    dichotomizes the dataset and split it into the features (X) and the labels (y).
    
    :return: two np.ndarray objects X and y.
    """
    data_dich = data.copy()
    data_dich['lab'] = data_dich['lab'].map({'NF': 0, 'B': 0, 'C': 0, 'M': 1, 'X': 1})
    y = data_dich['lab']
    X = data_dich.drop(['lab'], axis=1)
    return X.values, y.values

In [133]:
data_dir = ''
data_file = "toy_normalized_partition1ExtractedFeatures.csv"
data_file2 = "toy_normalized_partition2ExtractedFeatures.csv"

In [134]:
abt = pd.read_csv(os.path.join(data_dir, data_file))
abt2 = pd.read_csv(os.path.join(data_dir, data_file2))

---
### Q1 (10 points)

Just like you did with the previous assignment, you will be utilizing a few different types of feature selection to find subsets of descriptive features to use in the models we will be evaluating.  For this question you will again be utilizing the [SelectKBest](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest) class from [scikit-learn Univariate Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection). You will then be using 3 diferent feature evaluation functions.

-  [scikit-learn f_classif](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_classif.html#sklearn.feature_selection.f_classif)

- [scikit-learn mutual_info_classif](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_classif.html#sklearn.feature_selection.mutual_info_classif)

- [chi2](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.chi2.html#sklearn.feature_selection.chi2)

For each of these combinations of evaluation functions, you need to construct a 20 feature training and testing dataset. This will be done by:
<ol>
    <li>Use the `SelectKBest` class with each of the evaluation functions to perform feature selection using Partition 1 as your input data</li>
    <li>Construct a new train `DataFrame` for each instance of `SelectKBest` from 1 with the `lab` class labels using Partition 1</li>
    <li>Construct a new test `DataFrame` for each instance of `SelectKBest` from 1 with the `lab` class labels using Partition 2</li>
</ol>

After this question, you should have a total of 6 `DataFrame`s to use in later questions, a train and test pair for each feature selection method.

---

In [135]:
numFeat = 20
abt_cpy = abt.copy()
abt2_cpy = abt2.copy()

In [136]:
X_train = abt_cpy.drop(['lab'], axis=1)
y_train = abt_cpy['lab']

X_test = abt2_cpy.drop(['lab'], axis=1)
y_test = abt2_cpy['lab']
labels = pd.DataFrame(y_train, columns = ['lab'])
# f_classif
f_select = SelectKBest(f_classif, k=numFeat)
f_select.fit_transform(X_train.values,y_train.values)
col_index = f_select.get_support()
columns = X_train.columns[col_index]
f_train = abt_cpy[columns.values].join(y_train)
f_test = abt2_cpy[columns.values].join(y_test)
# mutual info classif
m_select = SelectKBest(mutual_info_classif, k=numFeat)
m_select.fit_transform(X_train.values,y_train.values)
col_index = m_select.get_support()
columns = X_train.columns[col_index]
m_train = abt_cpy[columns.values].join(y_train)
m_test = abt2_cpy[columns.values].join(y_test)
# chi2
c_select = SelectKBest(chi2, k=numFeat)
c_select.fit_transform(X_train.values,y_train.values)
col_index = m_select.get_support()
columns = X_train.columns[col_index]
c_train = abt_cpy[columns.values].join(y_train)
c_test = abt2_cpy[columns.values].join(y_test)

---
### Q2 (10 points)

Now that we have our training and testing datasets for each of our feature subsets, we need to attempt to perform hyperparameter tuning on our model for each of the datasets. We want to see which combination of dataset and parameter settings seem to provide the best results. 

In order to do this, we must first dichotomize the training and testing data. Lucky for you, a method has already been provided to do this. All you need to do is apply it to teach of the `DataFrame`s you constructed in Q1.  

With your binary classification dataset constructed, now it's time to start training and testing some models. We will start with the simple [KNeighborsClassifer](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html), and try several different settings to see how/if using different settings will improve our score. So, for each of your three copies of the Partition 1 training datasets that have had their `lab` columns converted to a binary label, train 4 different instances with the following settings. **(see documentation to know what these are)** In total you will train and evaluate 12 model setting and feature selected data pairings. 

|Model Number| n_neighbors | p |
|------------|-------------|---|
|1|3|1|
|2|3|2|
|3|5|1|
|4|5|2|


Once you have done that, test each of your models using your binary classification copy of the Partition 2 testing dataset that was cunstructed with the same features the model was trained on. You shall then calculate and print the TSS score for each result. **NOTE: The model does take a little while to evaluate.**

---

In [137]:
n_neighbors = [3, 5]
p = [1,2]
temp = [n_neighbors, p]
params = list(itertools.product(*temp))

In [138]:
# f_classif
f_train_X, f_train_y = dichotomize_X_y(f_train)
f_test_X, f_test_y = dichotomize_X_y(f_test)
print("f_classif score: ")
for n, pp in params:
    c = KNeighborsClassifier(n_neighbors=n, p=pp)
    c.fit(f_train_X, f_train_y)
    y_p = c.predict(f_test_X)
    score = calc_tss(f_test_y, y_p)
    print(score)
print("-------------------\n")

# mutual_info_classsif
m_train_X, m_train_y = dichotomize_X_y(m_train)
m_test_X, m_test_y = dichotomize_X_y(m_test)
print("mutual_info_classif score: ")
for n, pp in params:
    c = KNeighborsClassifier(n_neighbors=n, p=pp)
    c.fit(m_train_X, m_train_y)
    y_p = c.predict(m_test_X)
    score = calc_tss(m_test_y, y_p)
    print(score)
print("-------------------\n")

# chi2
c_train_X, c_train_y = dichotomize_X_y(c_train)
c_test_X, c_test_y = dichotomize_X_y(c_test)
print("chi2 score: ")
for n, pp in params:
    c = KNeighborsClassifier(n_neighbors=n, p=pp)
    c.fit(c_train_X, c_train_y)
    y_p = c.predict(c_test_X)
    score = calc_tss(c_test_y, y_p)
    print(score)
print("-------------------\n")

f_classif score: 
TN=52	FP=8	FN=3	TP=37
0.7916666666666667
TN=54	FP=6	FN=5	TP=35
0.775
TN=51	FP=9	FN=3	TP=37
0.775
TN=51	FP=9	FN=4	TP=36
0.75
-------------------

mutual_info_classif score: 
TN=51	FP=9	FN=7	TP=33
0.6749999999999999
TN=50	FP=10	FN=1	TP=39
0.8083333333333333
TN=51	FP=9	FN=6	TP=34
0.7
TN=52	FP=8	FN=3	TP=37
0.7916666666666667
-------------------

chi2 score: 
TN=51	FP=9	FN=7	TP=33
0.6749999999999999
TN=50	FP=10	FN=1	TP=39
0.8083333333333333
TN=51	FP=9	FN=6	TP=34
0.7
TN=52	FP=8	FN=3	TP=37
0.7916666666666667
-------------------



---
### Q3 (10 points)

After evaluating the various results from Q2, you will notice that the results are not all that great with greater than 1000 false negatives for nearly all of our settings tried. But, what can be done to improve our results? If you read the documentation for the [KNeighborsClassifer](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html), which you certainly should have, you will see that we were only using the `MinkowskiDistance` metric with different values of `p`. If you look into the [DistanceMetric](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html#sklearn.neighbors.DistanceMetric) documentation for the neighbors classifiers, you will see there are several others available to use.

So, for this question, train and evaluate two more instances of [KNeighborsClassifer](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) for each of our different feature selection train test datsets, but this time using the `ChebyshevDistance` metric instead of the `MinkowskiDistance` metric.  For these models you will only be changing the number neighbors to 3 and 5, as the values of `p` are not used for the `ChebyshevDistance` metric. 

---

In [139]:
n_neighbors = [3, 5]
temp = [n_neighbors]
params = list(itertools.product(*temp))

In [140]:
# f_classif
f_train_X, f_train_y = dichotomize_X_y(f_train)
f_test_X, f_test_y = dichotomize_X_y(f_test)
print("f_classif score: ")
for n in params:
    c = KNeighborsClassifier(n_neighbors=n[0], metric="chebyshev")
    c.fit(f_train_X, f_train_y)
    y_p = c.predict(f_test_X)
    score = calc_tss(f_test_y, y_p)
    print(score)
print("-------------------\n")

# mutual_info_classsif
m_train_X, m_train_y = dichotomize_X_y(m_train)
m_test_X, m_test_y = dichotomize_X_y(m_test)
print("mutual_info_classif score: ")
for n in params:
    c = KNeighborsClassifier(n_neighbors=n[0], metric="chebyshev")
    c.fit(m_train_X, m_train_y)
    y_p = c.predict(m_test_X)
    score = calc_tss(m_test_y, y_p)
    print(score)
print("-------------------\n")

# chi2
c_train_X, c_train_y = dichotomize_X_y(c_train)
c_test_X, c_test_y = dichotomize_X_y(c_test)
print("chi2 score: ")
for n in params:
    c = KNeighborsClassifier(n_neighbors=n[0], metric="chebyshev")
    c.fit(c_train_X, c_train_y)
    y_p = c.predict(c_test_X)
    score = calc_tss(c_test_y, y_p)
    print(score)
print("-------------------\n")

f_classif score: 
TN=53	FP=7	FN=5	TP=35
0.7583333333333333
TN=52	FP=8	FN=5	TP=35
0.7416666666666667
-------------------

mutual_info_classif score: 
TN=51	FP=9	FN=5	TP=35
0.725
TN=52	FP=8	FN=2	TP=38
0.8166666666666667
-------------------

chi2 score: 
TN=51	FP=9	FN=5	TP=35
0.725
TN=52	FP=8	FN=2	TP=38
0.8166666666666667
-------------------



---
### Q4 (10 points)

After evaluating the results from Q3, you will see that the results are no better than those we found for Q2. This leads to the thought that maybe the [KNeighborsClassifer](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) is just not a good fit for the problem we are applying it to. So, let's move on to another classifier for this problem. 

In this question, you will utilize the [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html), and try several different settings to see how/if using different settings will improve our score. So, continuing to use our training/testing pairs constructed with different feature selection methods that have had their `lab` column converted to a binary label, train 8 different instances with the following settings. **(see documentation to know what these are)**

|Model Number| criterion | max_depth | splitter |
|------------|---------|-------------|---|
|1|gini|5|best|
|2|gini|5|random|
|3|gini|None|best|
|4|gini|None|random|
|5|entropy|5|best|
|6|entropy|5|random|
|7|entropy|None|best|
|8|entropy|None|random|



Once you have done that, test each of your models using your binary classification copy of the Partition 2 testing dataset that was cunstructed with the same features the model was trained on. You shall then calculate and print the TSS score for each result.

---

In [141]:
criterion = ['gini', 'entropy']
depth = [5, None]
splitter = ['best', 'random']
temp = [criterion, depth, splitter]
params = list(itertools.product(*temp))

In [142]:
# f_classif
f_train_X, f_train_y = dichotomize_X_y(f_train)
f_test_X, f_test_y = dichotomize_X_y(f_test)
print("f_classif score: ")
for criteria, deep, split in params:
    c = DecisionTreeClassifier(criterion = criteria, max_depth = deep, 
                               splitter = split)
    c.fit(f_train_X, f_train_y)
    y_p = c.predict(f_test_X)
    score = calc_tss(f_test_y, y_p)
    print(score)
print("-------------------\n")

# mutual_info_classsif
m_train_X, m_train_y = dichotomize_X_y(m_train)
m_test_X, m_test_y = dichotomize_X_y(m_test)
print("mutual_info_classif score: ")
for criteria, deep, split in params:
    c = DecisionTreeClassifier(criterion = criteria, max_depth = deep, 
                               splitter = split)
    c.fit(m_train_X, m_train_y)
    y_p = c.predict(m_test_X)
    score = calc_tss(m_test_y, y_p)
    print(score)
print("-------------------\n")

# chi2
c_train_X, c_train_y = dichotomize_X_y(c_train)
c_test_X, c_test_y = dichotomize_X_y(c_test)
print("chi2 score: ")
for criteria, deep, split in params:
    c = DecisionTreeClassifier(criterion = criteria, max_depth = deep, 
                               splitter = split)
    c.fit(c_train_X, c_train_y)
    y_p = c.predict(c_test_X)
    score = calc_tss(c_test_y, y_p)
    print(score)
print("-------------------\n")

f_classif score: 
TN=47	FP=13	FN=2	TP=38
0.7333333333333333
TN=55	FP=5	FN=11	TP=29
0.6416666666666666
TN=50	FP=10	FN=4	TP=36
0.7333333333333334
TN=48	FP=12	FN=8	TP=32
0.6000000000000001
TN=47	FP=13	FN=3	TP=37
0.7083333333333334
TN=51	FP=9	FN=8	TP=32
0.65
TN=47	FP=13	FN=6	TP=34
0.6333333333333333
TN=50	FP=10	FN=4	TP=36
0.7333333333333334
-------------------

mutual_info_classif score: 
TN=50	FP=10	FN=16	TP=24
0.43333333333333335
TN=48	FP=12	FN=9	TP=31
0.575
TN=45	FP=15	FN=16	TP=24
0.35
TN=48	FP=12	FN=14	TP=26
0.45
TN=50	FP=10	FN=16	TP=24
0.43333333333333335
TN=47	FP=13	FN=15	TP=25
0.4083333333333333
TN=45	FP=15	FN=16	TP=24
0.35
TN=46	FP=14	FN=10	TP=30
0.5166666666666666
-------------------

chi2 score: 
TN=50	FP=10	FN=16	TP=24
0.43333333333333335
TN=54	FP=6	FN=16	TP=24
0.5
TN=49	FP=11	FN=16	TP=24
0.41666666666666663
TN=49	FP=11	FN=13	TP=27
0.4916666666666667
TN=45	FP=15	FN=16	TP=24
0.35
TN=48	FP=12	FN=15	TP=25
0.425
TN=46	FP=14	FN=16	TP=24
0.36666666666666664
TN=47	FP=13	FN=4	TP=36
0.68

---
### Q5 (10 points)

After evaluating results from Q4, you will see that the [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) was able to accomplish a bit of an improvement over the best resutls we found for the [KNeighborsClassifer](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html).  This is indeed great, but can we do better than this if we use yet another classifier? Let's move on to yet another and find out.

For this question you will be utilizing the [GaussianNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html) classifier. We won't be changing any of the default settings, just train 1 model for each of our feature selected data subsets. You will again be using your training/testing pairs constructed with different feature selection methods that have had their `lab` column converted to a binary label. You will then test each of your models using your binary classification copy of the Partition 2 testing dataset that was cunstructed with the same features the model was trained on. You shall then calculate and print the TSS score for each result.

---

In [143]:
# f_classif
f_train_X, f_train_y = dichotomize_X_y(f_train)
f_test_X, f_test_y = dichotomize_X_y(f_test)
print("f_classif score: ")
c = GaussianNB()
c.fit(f_train_X, f_train_y)
y_p = c.predict(f_test_X)
score = calc_tss(f_test_y, y_p)
print(score)
print("-------------------\n")

# mutual_info_classsif
m_train_X, m_train_y = dichotomize_X_y(m_train)
m_test_X, m_test_y = dichotomize_X_y(m_test)
print("mutual_info_classif score: ")
c = GaussianNB()
c.fit(m_train_X, m_train_y)
y_p = c.predict(m_test_X)
score = calc_tss(m_test_y, y_p)
print(score)
print("-------------------\n")

# chi2
c_train_X, c_train_y = dichotomize_X_y(c_train)
c_test_X, c_test_y = dichotomize_X_y(c_test)
print("chi2 score: ")
c = GaussianNB()
c.fit(c_train_X, c_train_y)
y_p = c.predict(c_test_X)
score = calc_tss(c_test_y, y_p)
print(score)
print("-------------------\n")

f_classif score: 
TN=41	FP=19	FN=1	TP=39
0.6583333333333333
-------------------

mutual_info_classif score: 
TN=38	FP=22	FN=2	TP=38
0.5833333333333333
-------------------

chi2 score: 
TN=38	FP=22	FN=2	TP=38
0.5833333333333333
-------------------



---
### Q6 (10 points)

If you recall from a lecture some time back, it was shown that another way of improving the results of classification is to perform some form of sampling to balance the number of samples there are for the various classes. The reason why this works for specific classifiers, and methods for doing the sampling, are numerious and we don't have enough time to cover all of them in this course.  However, it is still beneficial to know this works and that it is something that you should be considering when you are training models.  

So, for this question, we will implement a very naive method for sampling so we can use the results for training our models again.  Below you will find a function stub, complete the function and have it return a copy of the input dataframe where each class (except for the smallest one) have been undersampled to match the size of the smallest class in the dataset. In this function you should assume the `lab` column is the class label and not the dicotomized binary classification converted label.

To do this you may want to use the [groupby](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) function of the DataFrame to get groups of rows from your DataFrame.  You may also wish to use the [sample](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html) function to select a number of rows from a group. You can also use the [apply](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html) method to process each group from your grouped rows. These are just hints, you can solve the problem how you see fit.

Once this function is complete, apply it to each of your training datasets that have been constructed with different feature selection methods from partition 1 (the ones with all the NF, C, .., X labels). You will not be applying this to your testing sets. After you have your sampled feature selected datasets, you will then apply your function that converts the multi-class problem to a binary problem to each of the resultant selected subsets so we can use these new undersampled data for the next several questions.

---

In [156]:
def perform_under_sample(data:DataFrame)->DataFrame:
    new_dt = pd.DataFrame()
    data.groupby('lab')
    lab = data['lab']
    labels = list(set(lab.tolist()))
    vals = dict(lab.value_counts())
    m = vals[min(vals, key=vals.get)]
    for label in labels:
        s = data[lab == label]
        s = s.sample(n = m)
        new_dt = pd.concat([new_dt, s])
    
    return new_dt

In [157]:
f_train_under = perform_under_sample(f_train)
m_train_under = perform_under_sample(m_train)
c_train_under = perform_under_sample(c_train) 

---
### Q7

For this question repeat what you did for Q2, but with your balanced binary classification datasets constructed in Q6, uese the [KNeighborsClassifer](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html), and try several different settings to see how/if using different settings will improve our score. 

So, train 4 different instances with the following settings for each of your feature selected subsets, for a total of 12 different evaluations. **(see documentation to know what these are)**

|Model Number| n_neighbors | p |
|------------|-------------|---|
|1|3|1|
|2|3|2|
|3|5|1|
|4|5|2|


Once you have done that, test each of your models using your binary classification copy of Partition 2 testing dataset that was cunstructed with the same features the model was trained on (these should not have been balanced). You shall then calculate and print the TSS score for each result. **NOTE: The model now takes less time to evaluate!**

---

In [158]:
n_neighbors = [3, 5]
p = [1,2]
temp = [n_neighbors, p]
params = list(itertools.product(*temp))

In [159]:
# f_classif
f_train_X, f_train_y = dichotomize_X_y(f_train_under)
f_test_X, f_test_y = dichotomize_X_y(f_test)
print("f_classif score: ")
for n, pp in params:
    c = KNeighborsClassifier(n_neighbors=n, p=pp)
    c.fit(f_train_X, f_train_y)
    y_p = c.predict(f_test_X)
    score = calc_tss(f_test_y, y_p)
    print(score)
print("-------------------\n")

# mutual_info_classsif
m_train_X, m_train_y = dichotomize_X_y(m_train_under)
m_test_X, m_test_y = dichotomize_X_y(m_test)
print("mutual_info_classif score: ")
for n, pp in params:
    c = KNeighborsClassifier(n_neighbors=n, p=pp)
    c.fit(m_train_X, m_train_y)
    y_p = c.predict(m_test_X)
    score = calc_tss(m_test_y, y_p)
    print(score)
print("-------------------\n")

# chi2
c_train_X, c_train_y = dichotomize_X_y(c_train_under)
c_test_X, c_test_y = dichotomize_X_y(c_test)
print("chi2 score: ")
for n, pp in params:
    c = KNeighborsClassifier(n_neighbors=n, p=pp)
    c.fit(c_train_X, c_train_y)
    y_p = c.predict(c_test_X)
    score = calc_tss(c_test_y, y_p)
    print(score)
print("-------------------\n")

f_classif score: 
TN=52	FP=8	FN=3	TP=37
0.7916666666666667
TN=54	FP=6	FN=5	TP=35
0.775
TN=51	FP=9	FN=3	TP=37
0.775
TN=51	FP=9	FN=4	TP=36
0.75
-------------------

mutual_info_classif score: 
TN=51	FP=9	FN=7	TP=33
0.6749999999999999
TN=50	FP=10	FN=1	TP=39
0.8083333333333333
TN=51	FP=9	FN=6	TP=34
0.7
TN=52	FP=8	FN=3	TP=37
0.7916666666666667
-------------------

chi2 score: 
TN=51	FP=9	FN=7	TP=33
0.6749999999999999
TN=50	FP=10	FN=1	TP=39
0.8083333333333333
TN=51	FP=9	FN=6	TP=34
0.7
TN=52	FP=8	FN=3	TP=37
0.7916666666666667
-------------------



---
### Q8

After evaluating the various results from Q7, you will notice that some of the results are improved over the same experiments we conducted in Q2. Additionally, you should also notice a improvement in the speed at which the results were obtained. The question now is will we continue to see these improvements for all of our experiments? So, let's move on and see.

For this question, you will repeat the experiments from Q3, but using the balanced binary classification datasets constructed in Q6. You will still be using the [KNeighborsClassifer](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) like you did in Q7, but you will again be changing from using the `MinkowskiDistance` metric with different values of `p` to using the `ChebyshevDistance` metric. You will construct two models for each of your feature selected datasets by changing the number neighbors to 3 and 5.

Once you have done that, test each of your models using your binary classification copy of Partition 2 testing dataset that was cunstructed with the same features the model was trained on (these should not have been balanced), then calculate and print the TSS score for each result. 

---

In [160]:
n_neighbors = [3, 5]
temp = [n_neighbors]
params = list(itertools.product(*temp))

In [162]:
# f_classif
f_train_X, f_train_y = dichotomize_X_y(f_train_under)
f_test_X, f_test_y = dichotomize_X_y(f_test)
print("f_classif score: ")
for n in params:
    c = KNeighborsClassifier(n_neighbors=n[0], metric="chebyshev")
    c.fit(f_train_X, f_train_y)
    y_p = c.predict(f_test_X)
    score = calc_tss(f_test_y, y_p)
    print(score)
print("-------------------\n")

# mutual_info_classsif
m_train_X, m_train_y = dichotomize_X_y(m_train_under)
m_test_X, m_test_y = dichotomize_X_y(m_test)
print("mutual_info_classif score: ")
for n in params:
    c = KNeighborsClassifier(n_neighbors=n[0], metric="chebyshev")
    c.fit(m_train_X, m_train_y)
    y_p = c.predict(m_test_X)
    score = calc_tss(m_test_y, y_p)
    print(score)
print("-------------------\n")

# chi2
c_train_X, c_train_y = dichotomize_X_y(c_train_under)
c_test_X, c_test_y = dichotomize_X_y(c_test)
print("chi2 score: ")
for n in params:
    c = KNeighborsClassifier(n_neighbors=n[0], metric="chebyshev")
    c.fit(c_train_X, c_train_y)
    y_p = c.predict(c_test_X)
    score = calc_tss(c_test_y, y_p)
    print(score)
print("-------------------\n")

f_classif score: 
TN=53	FP=7	FN=5	TP=35
0.7583333333333333
TN=52	FP=8	FN=5	TP=35
0.7416666666666667
-------------------

mutual_info_classif score: 
TN=51	FP=9	FN=5	TP=35
0.725
TN=52	FP=8	FN=2	TP=38
0.8166666666666667
-------------------

chi2 score: 
TN=51	FP=9	FN=5	TP=35
0.725
TN=52	FP=8	FN=2	TP=38
0.8166666666666667
-------------------



---
### Q9

After evaluating the results of Q8 things are looking a little less encouraging, since none of those results look to be better than the results of Q7. However, the results from Q3 weren't really any better than Q2 in the first place, so not all is lost.  Let's continue on and see how things turn out with models like we used in Q4 since those were actaully an improvement over Q2 originally.

So, in this question, you will utilize the [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html), like you did in Q4, and try several different settings to see how/if using different settings will improve our score. The difference will again be that you are now using the balanced binary classification datasets constructed in Q6 to train 8 different instances for each of your feature selected datasets using the following settings. **(see documentation to know what these are)**

|Model Number| criterion | max_depth | splitter |
|------------|---------|-------------|---|
|1|gini|5|best|
|2|gini|5|random|
|3|gini|None|best|
|4|gini|None|random|
|5|entropy|5|best|
|6|entropy|5|random|
|7|entropy|None|best|
|8|entropy|None|random|



Once you have done that, test each of your models using your binary classification copy of copy of Partition 2 testing dataset that was cunstructed with the same features the model was trained on (this should not have been balanced), then calculate and print the TSS score for each result. 

---

In [150]:
criterion = ['gini', 'entropy']
depth = [5, None]
splitter = ['best', 'random']
temp = [criterion, depth, splitter]
params = list(itertools.product(*temp))

In [151]:
# f_classif
f_train_X, f_train_y = dichotomize_X_y(f_train_under)
f_test_X, f_test_y = dichotomize_X_y(f_test)
print("f_classif score: ")
for criteria, deep, split in params:
    c = DecisionTreeClassifier(criterion = criteria, max_depth = deep, 
                               splitter = split)
    c.fit(f_train_X, f_train_y)
    y_p = c.predict(f_test_X)
    score = calc_tss(f_test_y, y_p)
    print(score)
print("-------------------\n")

# mutual_info_classsif
m_train_X, m_train_y = dichotomize_X_y(m_train_under)
m_test_X, m_test_y = dichotomize_X_y(m_test)
print("mutual_info_classif score: ")
for criteria, deep, split in params:
    c = DecisionTreeClassifier(criterion = criteria, max_depth = deep, 
                               splitter = split)
    c.fit(m_train_X, m_train_y)
    y_p = c.predict(m_test_X)
    score = calc_tss(m_test_y, y_p)
    print(score)
print("-------------------\n")

# chi2
c_train_X, c_train_y = dichotomize_X_y(c_train_under)
c_test_X, c_test_y = dichotomize_X_y(c_test)
print("chi2 score: ")
for criteria, deep, split in params:
    c = DecisionTreeClassifier(criterion = criteria, max_depth = deep, 
                               splitter = split)
    c.fit(c_train_X, c_train_y)
    y_p = c.predict(c_test_X)
    score = calc_tss(c_test_y, y_p)
    print(score)
print("-------------------\n")

f_classif score: 
TN=46	FP=14	FN=5	TP=35
0.6416666666666666
TN=50	FP=10	FN=5	TP=35
0.7083333333333334
TN=49	FP=11	FN=4	TP=36
0.7166666666666667
TN=46	FP=14	FN=7	TP=33
0.5916666666666666
TN=46	FP=14	FN=3	TP=37
0.6916666666666667
TN=51	FP=9	FN=4	TP=36
0.75
TN=47	FP=13	FN=7	TP=33
0.6083333333333333
TN=50	FP=10	FN=6	TP=34
0.6833333333333333
-------------------

mutual_info_classif score: 
TN=47	FP=13	FN=16	TP=24
0.3833333333333333
TN=51	FP=9	FN=8	TP=32
0.65
TN=50	FP=10	FN=16	TP=24
0.43333333333333335
TN=53	FP=7	FN=13	TP=27
0.5583333333333333
TN=45	FP=15	FN=16	TP=24
0.35
TN=48	FP=12	FN=4	TP=36
0.7
TN=49	FP=11	FN=16	TP=24
0.41666666666666663
TN=46	FP=14	FN=12	TP=28
0.4666666666666666
-------------------

chi2 score: 
TN=50	FP=10	FN=16	TP=24
0.43333333333333335
TN=41	FP=19	FN=7	TP=33
0.5083333333333333
TN=47	FP=13	FN=16	TP=24
0.3833333333333333
TN=51	FP=9	FN=18	TP=22
0.4
TN=46	FP=14	FN=16	TP=24
0.36666666666666664
TN=52	FP=8	FN=5	TP=35
0.7416666666666667
TN=49	FP=11	FN=16	TP=24
0.416666666666

---
### Q10

Unlike with [KNeighborsClassifer](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html), it seems that the sampling didn't really help much for the [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html).  Where before we saw a 3X improvement with the Decision Tree over the KNN classifier, we now see similar results for both classifiers.  Let's see how switching to the sampled data affectes our best performing classifier when we were using the full dataset.

For this question you will again be utilizing the [GaussianNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html) classifier as you did in Q5 but using your balanced binary classification dataset constructed in Q6 to train just 1 model for each feature selected dataset. Once you have done that, test the model using your binary classification copy of Partition 2 testing dataset that was cunstructed with the same features the model was trained on (this should not have been balanced), then calculate and print the TSS score. 

---

In [152]:
# f_classif
f_train_X, f_train_y = dichotomize_X_y(f_train_under)
f_test_X, f_test_y = dichotomize_X_y(f_test)
print("f_classif score: ")
c = GaussianNB()
c.fit(f_train_X, f_train_y)
y_p = c.predict(f_test_X)
score = calc_tss(f_test_y, y_p)
print(score)
print("-------------------\n")

# mutual_info_classsif
m_train_X, m_train_y = dichotomize_X_y(m_train_under)
m_test_X, m_test_y = dichotomize_X_y(m_test)
print("mutual_info_classif score: ")
c = GaussianNB()
c.fit(m_train_X, m_train_y)
y_p = c.predict(m_test_X)
score = calc_tss(m_test_y, y_p)
print(score)
print("-------------------\n")

# chi2
c_train_X, c_train_y = dichotomize_X_y(c_train_under)
c_test_X, c_test_y = dichotomize_X_y(c_test)
print("chi2 score: ")
c = GaussianNB()
c.fit(c_train_X, c_train_y)
y_p = c.predict(c_test_X)
score = calc_tss(c_test_y, y_p)
print(score)
print("-------------------\n")

f_classif score: 
TN=41	FP=19	FN=1	TP=39
0.6583333333333333
-------------------

mutual_info_classif score: 
TN=38	FP=22	FN=2	TP=38
0.5833333333333333
-------------------

chi2 score: 
TN=38	FP=22	FN=2	TP=38
0.5833333333333333
-------------------



Unfortunately, we don't see much improvement for our [GaussianNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html) classifier. 

**Note:The TA would like you to turn in assignments that have been run and have results, so make sure to do a restart and run all from the kernel menu. Then make sure to save before you turn it in. You might find it necessary to use the toy dataset if you have time constraints.**